### PROJECT 1 - Artificial Vision

In [70]:
    #"translation",
    # "erosion",
    #"dilation",
    #"closing",
    #"opening",

In [71]:
# ==============================
# 📌 IMPORT LIBRARIES 
# ==============================

import cv2
import numpy as np
import os
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
import time
from collections import deque

print("✅ Libraries loaded!")

✅ Libraries loaded!


In [ ]:


# ==============================
# 📌 DEFINE VARIABLES AT THE TOP
# ==============================

# Image Path
IMG_PATH = r"C:\Users\oleru\OneDrive - NTNU\08___4klasse_UPV\04__VA_Artificial_Vision\03__Artificial_Vision_Project\AV_Artificial_Vision_Project\01__Project1\imgs\spirals.bmp"

# Morphological Kernel
KERNEL = np.ones((5, 5), np.uint8)

# Transformation Parameters
TRANSLATION_X, TRANSLATION_Y = 0, 0
THRESHOLD_VALUE = 50
RESIZE_DIM = (300, 300)

# Morphological Operation Strength
EROSION_ITERATIONS = 1
DILATION_ITERATIONS = 1

# Median Filter Kernel Size (Must be an odd number)
MEDIAN_KERNEL_SIZE = 5

# Object Labeling Size Range
LABEL_MIN_SIZE = 100
LABEL_MAX_SIZE = 15000

# Extract image name without extension
img_filename = os.path.basename(IMG_PATH)
img_name, img_ext = os.path.splitext(img_filename)
output_filename = f"Final_{img_name}{img_ext}"
output_path = os.path.join(os.path.dirname(IMG_PATH), output_filename)

# Define processing sequence
PROCESSING_ORDER = [
    "median_filter",
    "grayscale",
    "thresholding",
    "invert_colors",                         # If object of interest = white
    #"labeling_slow",                        #connected_components_algorithm
    #"labelling_fast",                       #connected_components_algorithm more efficient
    #"labelling_edge_detection",             #a smart and efficient way
    "labelling_cv2",                        #using the cv2 connected_components library
    #"BFS_flood_fill"                         #BFS flood fill, much faster   
]


# ==============================
# 📌 DEFINE IMAGE PROCESSING FUNCTIONS
# ==============================

def translate(image):
    h, w = image.shape[:2]
    translation_matrix = np.float32([[1, 0, TRANSLATION_X], [0, 1, TRANSLATION_Y]])
    return cv2.warpAffine(image, translation_matrix, (w, h))


def median_filter(image):
    return cv2.medianBlur(image, MEDIAN_KERNEL_SIZE)


def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


def thresholding(image):
    if len(image.shape) == 3:
        image = grayscale(image)
    _, binary_img = cv2.threshold(image, THRESHOLD_VALUE, 255, cv2.THRESH_BINARY)
    return binary_img


def morph_opening(image):
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, KERNEL)


def erosion(image):
    return cv2.erode(image, KERNEL, iterations=EROSION_ITERATIONS)


def dilation(image):
    return cv2.dilate(image, KERNEL, iterations=DILATION_ITERATIONS)


def morph_closing(image):
    return cv2.morphologyEx(image, cv2.MORPH_CLOSE, KERNEL)


def edge_detection(image):
    edges = cv2.Canny(image, 100, 200)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    edge_highlighted = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR) if len(image.shape) == 2 else image
    cv2.drawContours(edge_highlighted, contours, -1, (0, 255, 0), 2)
    return edge_highlighted


def label_components(image):
    image_gray = grayscale(image) if len(image.shape) == 3 else image
    inverted_img = cv2.bitwise_not(image_gray)
    _, binary_img = cv2.threshold(inverted_img, 50, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    labeled_img = cv2.cvtColor(binary_img, cv2.COLOR_GRAY2BGR)

    object_count = 0
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = w * h
        if LABEL_MIN_SIZE <= area <= LABEL_MAX_SIZE:
            object_count += 1
            cv2.rectangle(labeled_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(labeled_img, f"ID {object_count} ({area}px)", (x, y - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    return labeled_img


def convert_to_bgr(image):
    return cv2.cvtColor(image, cv2.COLOR_GRAY2BGR) if len(image.shape) == 2 else image


def resize_image(image, size=RESIZE_DIM):
    return cv2.resize(image, size)

def labeling_cv2(image):

    binary = (image == 0).astype(np.uint8)

    # Use OpenCV's optimized connectedComponents function
    N, labeled_img = cv2.connectedComponents(binary, connectivity=8)

    rows, cols = labeled_img.shape

    # Add random colour to each object
    output = np.zeros((rows, cols, 3), dtype=np.uint8)
    colors = {0: (0, 0, 0)}
    for l in range(1, N + 1):
        colors[l] = (np.random.randint(0, 256),
                     np.random.randint(0, 256),
                     np.random.randint(0, 256))
    for i in range(rows):
        for j in range(cols):
            output[i, j] = colors[labeled_img[i, j]]
    
    # Make boxes around found objects
    valid_components = 0
    for label in range(1, N + 1):
        coords = np.column_stack(np.where(labeled_img == label))
        if coords.size == 0:
            continue

        top, left = int(coords[:, 0].min()), int(coords[:, 1].min())
        bottom, right = int(coords[:, 0].max()), int(coords[:, 1].max())
        bbox_area = (right - left) * (bottom - top)

        #check if the object is without bounds
        if LABEL_MIN_SIZE <= bbox_area <= LABEL_MAX_SIZE:
            valid_components += 1
            cv2.rectangle(output, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(output, f"ID {label} ({bbox_area}px)", (left, top - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    
    # Display number of objects
    cv2.putText(output, f"Total Components: {valid_components}", (10, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    return output



def labeling_cc(image):
    # A = binary picture
    # P = labeled_img, starting blank

    binary = (image == 0).astype(np.uint8)
    rows, cols = binary.shape
    labeled_img = np.zeros((rows, cols), dtype=np.int32)

    # 8-connectivity neighbors
    neighbors = [(-1, -1), (-1, 0), (-1, 1),
                 (0, -1),           (0, 1),
                 (1, -1),  (1, 0),  (1, 1)]
    
    N = 0      
    found = True
    while found:
        found = False

        # SEARCH
        for i in range(rows):
            for j in range(cols):
                if binary[i, j] == 1 and labeled_img[i, j] == 0:
                    N += 1            
                    labeled_img[i, j] = N  
                    found = True
                    break
            if found:
                break
        
        # PROPAGATION
        if found:
            finished = False
            while not finished:
                finished = True 
                for i in range(rows):
                    for j in range(cols):
                        if binary[i, j] == 1 and labeled_img[i, j] == 0:
                            for di, dj in neighbors:
                                ni, nj = i + di, j + dj  

                                if 0 <= ni < rows and 0 <= nj < cols:  
                                    if labeled_img[ni, nj] == N: 
                                        labeled_img[i, j] = N 
                                        finished = False 
                                        break 


    # Add random colour to each object
    output = np.zeros((rows, cols, 3), dtype=np.uint8)
    colors = {0: (0, 0, 0)}
    for l in range(1, N + 1):
        colors[l] = (np.random.randint(0, 256),
                     np.random.randint(0, 256),
                     np.random.randint(0, 256))
    for i in range(rows):
        for j in range(cols):
            output[i, j] = colors[labeled_img[i, j]]
    

    # Make boxes around found objects
    valid_components = 0
    for label in range(1, N + 1):
        coords = np.column_stack(np.where(labeled_img == label))
        if coords.size == 0:
            continue

        top, left = int(coords[:, 0].min()), int(coords[:, 1].min())
        bottom, right = int(coords[:, 0].max()), int(coords[:, 1].max())
        bbox_area = (right - left) * (bottom - top)

        #check if the object is without bounds
        if LABEL_MIN_SIZE <= bbox_area <= LABEL_MAX_SIZE:
            valid_components += 1
            cv2.rectangle(output, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(output, f"ID {label} ({bbox_area}px)", (left, top - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    
    # Display number of objects
    cv2.putText(output, f"Total Components: {valid_components}", (10, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    return output

def labeling_cc_optimized(image):
    # A = binary picture
    # P = labeled_img, starting blank

    binary = (image == 0).astype(np.uint8)
    rows, cols = binary.shape
    labeled_img = np.zeros((rows, cols), dtype=np.int32)

    # 8-connectivity neighbors
    neighbors = [(-1, -1), (-1, 0), (-1, 1),
                 (0, -1),           (0, 1),
                 (1, -1),  (1, 0),  (1, 1)]
    
    N = 0      
    found = True
    while found:
        found = False

        # SEARCH
        for i in range(rows):
            for j in range(cols):
                if binary[i, j] == 1 and labeled_img[i, j] == 0:
                    N += 1
                    queue = deque([(i, j)])            
                    labeled_img[i, j] = N  
                    found = True
                    break
            if found:
                break
        
        # PROPAGATION IN FOUR-QUADRANT
        if found:
            finished = False
            while not finished:
                finished = True

                # IT1 -> Top-left to bottom-right 
                for i in range(rows):
                    for j in range(cols):
                        if binary[i, j] == 1 and labeled_img[i, j] == 0:
                            for di, dj in neighbors:
                                ni, nj = i + di, j + dj  

                                if 0 <= ni < rows and 0 <= nj < cols:  
                                    if labeled_img[ni, nj] == N: 
                                        labeled_img[i, j] = N 
                                        finished = False 
                                        break 

                # IT2 -> Bottom-right to top-left
                for i in range(rows - 1, -1, -1):
                    for j in range(cols - 1, -1, -1):
                        if binary[i, j] == 1 and labeled_img[i, j] == 0:
                            for di, dj in neighbors:
                                ni, nj = i + di, j + dj  

                                if 0 <= ni < rows and 0 <= nj < cols:  
                                    if labeled_img[ni, nj] == N: 
                                        labeled_img[i, j] = N 
                                        finished = False 
                                        break 
                
                # IT3 -> Bottom-left to top-right
                for i in range(rows - 1, -1, -1):
                    for j in range(cols):
                        if binary[i, j] == 1 and labeled_img[i, j] == 0:
                            for di, dj in neighbors:
                                ni, nj = i + di, j + dj  

                                if 0 <= ni < rows and 0 <= nj < cols:  
                                    if labeled_img[ni, nj] == N: 
                                        labeled_img[i, j] = N 
                                        finished = False 
                                        break 
                
                # IT4 -> Top-right to bottom-left
                for i in range(rows):
                    for j in range(cols -1, -1, -1):
                        if binary[i, j] == 1 and labeled_img[i, j] == 0:
                            for di, dj in neighbors:
                                ni, nj = i + di, j + dj  

                                if 0 <= ni < rows and 0 <= nj < cols:  
                                    if labeled_img[ni, nj] == N: 
                                        labeled_img[i, j] = N 
                                        finished = False 
                                        break 
                                        

    # Add random colour to each object
    output = np.zeros((rows, cols, 3), dtype=np.uint8)
    colors = {0: (0, 0, 0)}
    for l in range(1, N + 1):
        colors[l] = (np.random.randint(0, 256),
                     np.random.randint(0, 256),
                     np.random.randint(0, 256))
    for i in range(rows):
        for j in range(cols):
            output[i, j] = colors[labeled_img[i, j]]
    

    # Make boxes around found objects
    valid_components = 0
    for label in range(1, N + 1):
        coords = np.column_stack(np.where(labeled_img == label))
        if coords.size == 0:
            continue

        top, left = int(coords[:, 0].min()), int(coords[:, 1].min())
        bottom, right = int(coords[:, 0].max()), int(coords[:, 1].max())
        bbox_area = (right - left) * (bottom - top)

        #check if the object is without bounds
        if LABEL_MIN_SIZE <= bbox_area <= LABEL_MAX_SIZE:
            valid_components += 1
            cv2.rectangle(output, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(output, f"ID {label} ({bbox_area}px)", (left, top - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    
    # Display number of objects
    cv2.putText(output, f"Total Components: {valid_components}", (10, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    return output


def BFS_flood_fill(image):
    binary = (image == 0).astype(np.uint8)
    rows, cols = binary.shape
    labeled_img = np.zeros((rows, cols), dtype=np.int32)

    # 8-connectivity neighbors for each quadrant 
    quadrant_neighbors = [
        [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)],  # Top-left to bottom-right
        [(1, 1), (1, 0), (1, -1), (0, 1), (0, -1), (-1, 1), (-1, 0), (-1, -1)],  # Bottom-right to top-left
        [(1, -1), (1, 0), (1, 1), (0, -1), (0, 1), (-1, -1), (-1, 0), (-1, 1)],  # Bottom-left to top-right
        [(-1, 1), (-1, 0), (-1, -1), (0, 1), (0, -1), (1, 1), (1, 0), (1, -1)]   # Top-right to bottom-left
    ]

    N = 0
    for i in range(rows):
        for j in range(cols):
            if binary[i, j] == 1 and labeled_img[i, j] == 0:
                N += 1
                queue = deque([(i, j)])
                labeled_img[i, j] = N

                # BFS
                for neighbors in quadrant_neighbors:
                    local_queue = deque(queue)  # Using a queue
                    while local_queue:
                        x, y = local_queue.popleft()
                        for dx, dy in neighbors:
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < rows and 0 <= ny < cols and binary[nx, ny] == 1 and labeled_img[nx, ny] == 0:
                                labeled_img[nx, ny] = N
                                local_queue.append((nx, ny))


    # Add random colour to each object
    output = np.zeros((rows, cols, 3), dtype=np.uint8)
    colors = {0: (0, 0, 0)}
    for l in range(1, N + 1):
        colors[l] = (np.random.randint(0, 256),
                     np.random.randint(0, 256),
                     np.random.randint(0, 256))
    for i in range(rows):
        for j in range(cols):
            output[i, j] = colors[labeled_img[i, j]]
    

    # Make boxes around found objects
    valid_components = 0
    for label in range(1, N + 1):
        coords = np.column_stack(np.where(labeled_img == label))
        if coords.size == 0:
            continue

        top, left = int(coords[:, 0].min()), int(coords[:, 1].min())
        bottom, right = int(coords[:, 0].max()), int(coords[:, 1].max())
        bbox_area = (right - left) * (bottom - top)

        #check if the object is without bounds
        if LABEL_MIN_SIZE <= bbox_area <= LABEL_MAX_SIZE:
            valid_components += 1
            cv2.rectangle(output, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(output, f"ID {label} ({bbox_area}px)", (left, top - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    
    # Display number of objects
    cv2.putText(output, f"Total Components: {valid_components}", (10, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    return output

def invert_colors(image):
    return cv2.bitwise_not(image)

def measure_time(func, *args):
    start_time = time.time()
    result = func(*args)
    elapsed_time = (time.time() - start_time) * 1000  # Convert to milliseconds
    return result, elapsed_time

# ==============================
# 📌 PROCESS IMAGE SEQUENTIALLY
# ==============================

img = cv2.imread(IMG_PATH)
if img is None:
    print("Error: Image could not be loaded. Please check the file path!")
    exit()
else:
    print("✅ Image loaded successfully!")

processed_image = img
step_images = [convert_to_bgr(resize_image(img))]
titles = ["Original Image"]

for step in PROCESSING_ORDER:
    if step == "translation":
        processed_image = translate(processed_image)
    elif step == "median_filter":
        processed_image = median_filter(processed_image)
    elif step == "grayscale":
        processed_image = grayscale(processed_image)
    elif step == "thresholding":
        processed_image = thresholding(processed_image)
    elif step == "opening":
        processed_image = morph_opening(processed_image)
    elif step == "erosion":
        processed_image = erosion(processed_image)
    elif step == "dilation":
        processed_image = dilation(processed_image)
    elif step == "closing":
        processed_image = morph_closing(processed_image)
    elif step == "invert_colors":
        processed_image, elapsed_time = measure_time(invert_colors, processed_image)
    elif step == "labeling_slow":
        start_time = time.time()

        processed_image = labeling_cc(processed_image)

        end_time = time.time()
        print(f"Connected-component algorithm completed in {end_time - start_time:.4f} seconds.")

    elif step == "labelling_fast":
        start_time = time.time()

        processed_image = labeling_cc_optimized(processed_image)
        
        end_time = time.time()
        print(f"Connected-component algorithm (multi-direction) completed in {end_time - start_time:.4f} seconds.")

    elif step == "labelling_edge_detection":
        start_time = time.time()

        processed_image = edge_detection(processed_image)
        processed_image = label_components(processed_image)
        
        end_time = time.time()
        print(f"Edge detection by contour completed in {end_time - start_time:.4f} seconds.")
    
    elif step == "labelling_cv2":
        start_time = time.time()

        processed_image = labeling_cv2(processed_image)
        
        end_time = time.time()
        print(f"Edge detection by CV2 completed in {end_time - start_time:.4f} seconds.")

    elif step == "BFS_flood_fill":
        start_time = time.time()

        processed_image = BFS_flood_fill(processed_image)
        
        end_time = time.time()
        print(f"Edge detection by BFS flood fill completed in {end_time - start_time:.4f} seconds.")


    step_images.append(convert_to_bgr(resize_image(processed_image)))
    titles.append(step.replace("_", " ").capitalize())

# ==============================
# 📌 DISPLAY RESULTS IN GRID
# ==============================

fig, axes = plt.subplots(
    nrows=2,
    ncols=(len(step_images) + 1) // 2,
    figsize=(15, 8)
)

axes = axes.flatten()

for i, (img, title) in enumerate(zip(step_images, titles)):
    axes[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[i].set_title(title)
    axes[i].axis("off")

plt.tight_layout()
plt.show()

# ==============================
# 📌 SAVE FINAL IMAGE
# ==============================

cv2.imwrite(output_path, processed_image)
print(f"✅ Final image saved as: {output_path}")

✅ Image loaded successfully!
Edge detection by CV2 completed in 0.0284 seconds.
✅ Final image saved as: C:\Users\oleru\OneDrive - NTNU\08___4klasse_UPV\04__VA_Artificial_Vision\03__Artificial_Vision_Project\AV_Artificial_Vision_Project\01__Project1\imgs\Final_spirals.bmp
